In [1]:
import pandas as pd
import numpy as np

from statsmodels.graphics.tsaplots import plot_acf, acf

/Users/idris/miniconda3/envs/forecast/lib/python3.8/site-packages/statsmodels/compat/pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [3]:
path = '/Users/idris/Documents/ds_project/forecast_store_sales/'

df = pd.read_csv(path + 'data/trainclean.csv', sep=';', parse_dates=['date'], index_col=['date'])

df.head()

,ts_id,id,bu,famid,sales,onprom
date,,,,,,
2013-01-02,1_0,1782,1,0,2.0,0
2013-01-03,1_0,3564,1,0,3.0,0
2013-01-04,1_0,5346,1,0,3.0,0
2013-01-05,1_0,7128,1,0,5.0,0
2013-01-06,1_0,8910,1,0,2.0,0


In [4]:
df_pivot = df.pivot_table(index='date', columns='ts_id', values='sales')
df_pivot.head()

ts_id,10_0,10_1,10_10,10_11,10_12,10_13,10_14,10_15,10_16,10_17,...,9_29,9_3,9_30,9_31,9_32,9_5,9_6,9_7,9_8,9_9
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-02,3.0,NaN,57.0,41.164,2579.0,8.0,2.0,NaN,NaN,NaN,...,83.0,1309.0,NaN,NaN,29.214,1001.0,NaN,2624.0,473.0,725.902
2013-01-03,2.0,NaN,38.0,14.470,1984.0,1.0,0.0,NaN,NaN,NaN,...,66.0,1047.0,NaN,NaN,25.000,747.0,NaN,1781.0,231.0,414.087
2013-01-04,2.0,NaN,22.0,10.691,1565.0,2.0,0.0,NaN,NaN,NaN,...,57.0,986.0,NaN,NaN,11.000,513.0,NaN,1383.0,205.0,238.125
2013-01-05,0.0,NaN,43.0,13.891,2544.0,5.0,3.0,NaN,NaN,NaN,...,84.0,1415.0,NaN,NaN,28.361,631.0,NaN,2259.0,416.0,472.871


In [5]:
l_lag = []
df_col = df_pivot.columns

for ts in df_col:
    
    afc_array = acf(df_pivot[ts], nlags=360, missing='drop')
    lag_array = afc_array.argsort()[-11:][::-1]
    lag_array = np.delete(lag_array, 0)
    lag_array = np.sort(lag_array)
    l_lag.append(lag_array)
    
df_lag = pd.DataFrame(l_lag).T
df_lag.columns = df_col

df_lag.head()

ts_id,10_0,10_1,10_10,10_11,10_12,10_13,10_14,10_15,10_16,10_17,...,9_29,9_3,9_30,9_31,9_32,9_5,9_6,9_7,9_8,9_9
0,1,1,7,1,7,1,1,1,1,5,...,7,1,1,1,1,7,1,28,7,7
1,4,2,14,2,14,2,5,2,2,20,...,14,7,2,2,7,14,6,63,14,14
2,7,3,21,3,28,6,7,3,3,21,...,21,14,5,3,14,21,7,91,21,28
3,13,4,28,4,63,7,8,4,4,32,...,28,21,6,4,21,28,8,119,28,35
4,14,5,35,5,91,14,11,5,5,35,...,35,28,7,5,28,35,13,154,35,56


In [6]:
df_lag_export = df_lag.melt(var_name='ts_id', value_name='lag')
df_lag_export.head()

,ts_id,lag
0,10_0,1
1,10_0,4
2,10_0,7
3,10_0,13
4,10_0,14


In [7]:
df_lag_export.to_csv('data/specific_lag_ts.csv', index=False)